In [1]:
import glob
import os

import pandas as pd
from tqdm import tqdm

work_path = os.path.abspath(os.getcwd())
project_name = "saturday7_KRX_HFT"
project_path = work_path[:work_path.find(project_name) + len(project_name)]
os.chdir(project_path)

In [2]:
# 파일시스템: date > code
# pandas key: (nsYMD, nsISIN)
date_files: list = glob.glob("./data/krx_raw/*")
pandas_keys = []

df_info = pd.DataFrame()
for date_file in tqdm(date_files):
    date = date_file.split("/")[-1]
    isin_files: list = glob.glob(date_file + "/*")

    isin_info = {}
    for isin_file in isin_files:
        isin = isin_file.split("/")[-1].split("_")[0]
        d_level = isin_file.split("/")[-1].split("_")[1].split("@")[0]

        isin_info[isin] = isin_info.get(isin, {})
        isin_info[isin].setdefault('A3', False)
        isin_info[isin].setdefault('B6', False)
        isin_info[isin].setdefault('A3_len', 0)
        isin_info[isin].setdefault('A3_len', 0)

        data_length = pd.read_parquet(isin_file).shape[0]
        if d_level == "A3":
            isin_info[isin]['A3_len'] = data_length
        elif d_level == "B6":
            isin_info[isin]['B6_len'] = data_length

        isin_info[isin]['A3'] = True if d_level == "A3" else isin_info[isin]['A3']
        isin_info[isin]['B6'] = True if d_level == "B6" else isin_info[isin]['B6']

    df_isin_info = pd.DataFrame(isin_info).T
    df_isin_info.index.name = 'nsISIN'
    df_isin_info['nsYMD'] = date
    df_info = pd.concat([df_info, df_isin_info])

df_info = df_info.reset_index()
df_info["nsStockCode"] = df_info["nsISIN"].apply(lambda x: 'A' + x[3:-3])
df_info = df_info.set_index(['nsYMD', 'nsStockCode'])

100%|██████████| 3/3 [00:01<00:00,  2.25it/s]


In [12]:
# df_krx_ohlcv_part = pd.read_parquet("./data/krx_mst/krx_ohlcv_part.parquet")
# df_krx_mst = pd.merge(df_info, df_krx_ohlcv_part, how="left", on=["nsYMD", "nsStockCode"])
# df_krx_vwap_part = pd.read_parquet("./data/krx_mst/krx_vwap_part.parquet")
# df_krx_mst = pd.merge(df_krx_mst, df_krx_vwap_part, how="left", on=["nsYMD", "nsStockCode"])
df_krx_mst = df_info.copy()
df_krx_mst = df_krx_mst.sort_values(by=['nsYMD', 'nsStockCode'],ascending=[True, True])
df_krx_mst = df_krx_mst.reset_index(drop=False)
df_krx_mst = df_krx_mst.query("A3_len != 0 or B6_len != 0")
df_krx_mst.to_parquet("./data/krx_mst/krx_mst.parquet", compression='zstd')
print(df_krx_mst.shape)

(15, 7)


In [13]:
df_krx_mst

,nsYMD,nsStockCode,nsISIN,A3,B6,A3_len,B6_len
0,20221004,A000270,KR7000270009,True,True,18736,46515
1,20221004,A000660,KR7000660001,True,True,46264,74380
2,20221004,A032190,KR7032190001,True,True,1394,4714
3,20221004,A035420,KR7035420009,True,True,121848,183015
4,20221004,A373220,KR7373220003,True,True,25422,54240
5,20221005,A000270,KR7000270009,True,True,49992,83665
6,20221005,A000660,KR7000660001,True,True,66785,102622
7,20221005,A032190,KR7032190001,True,True,763,3621
8,20221005,A035420,KR7035420009,True,True,112563,137010
9,20221005,A373220,KR7373220003,True,True,37070,68833
